## Synthetic data

In [1]:
%load_ext autoreload 
%autoreload 2

import sys
import ast

import pandas as pd
import numpy as np

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
from src import aposteriori

import tasks.run_helper

In [2]:
def parse_nullable_int_list(cell):
    try:
        parsed = ast.literal_eval(cell)
        return [int(x) if pd.notna(x) else pd.NA for x in parsed]
    except (ValueError, SyntaxError, TypeError, NameError):
        return [pd.NA]  # or [] if you prefer empty lists for invalid input


def str_list_to_list(str_list: str) -> list[str]:
    ls = str_list[1:-1].split(",")
    ls = [x.strip() if x is not None else x for x in ls]
    return ls


syn_df = pd.read_csv("../data/synthetic.csv")
syn_df["comment_key"] = syn_df.message + syn_df.conv_id

syn_df.Toxicity = syn_df.Toxicity.apply(str_list_to_list).apply(
    lambda x: [int(tox) for tox in x]
)

syn_df.age_annot = syn_df.age_annot.apply(parse_nullable_int_list).apply(
    lambda x : pd.cut(x, bins=4)
)
syn_df.sex_annot = syn_df.sex_annot.apply(str_list_to_list)
syn_df.demographic_group_annot = syn_df.demographic_group_annot.apply(
    str_list_to_list

syn_df

,conv_id,message,intent,Toxicity,Argument Quality,age_user,sex_user,sexual_orientation_user,demographic_group_user,current_employment_user,education_level_user,age_annot,sex_annot,sexual_orientation_annot,demographic_group_annot,current_employment_annot,education_level_annot,comment_key
0,028f869d-cf12-444a-aefc-62f8b6294acd,@FitnessJunkie07 Respect and honesty are unive...,"['Neutral', 'Neutral', 'Neutral', 'Neutral', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1]","['25', '25', '25', '25', '25', '25', '25', '25...","['male', 'male', 'male', 'male', 'male', 'male...","['Bisexual', 'Bisexual', 'Bisexual', 'Bisexual...","['Asian', 'Asian', 'Asian', 'Asian', 'Asian', ...","['Software Developer', 'Software Developer', '...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","[(20.956, 32.0], (20.956, 32.0], (20.956, 32.0...","['female', 'male', 'male', 'male', 'male', 'ma...","['Lesbian', 'Pansexual', 'Homosexual', 'Hetero...","['Arab', 'White', 'Caucasian', 'South Asian', ...","['Poet', 'Game Developer', 'Research Scientist...","[""Master's"", 'Some College', ""Bachelor's"", 'Ph...",@FitnessJunkie07 Respect and honesty are unive...
1,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I agree, but sometimes chang...","['Neutral', 'Neutral', 'Neutral', 'Neutral', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","['27', '27', '27', '27', '27', '27', '27', '27...","['female', 'female', 'female', 'female', 'fema...","['Lesbian', 'Lesbian', 'Lesbian', 'Lesbian', '...","['Arab', 'Arab', 'Arab', 'Arab', 'Arab', 'Arab...","['Poet', 'Poet', 'Poet', 'Poet', 'Poet', 'Poet...","[""Master's"", ""Master's"", ""Master's"", ""Master's...","[(20.956, 32.0], (20.956, 32.0], (20.956, 32.0...","['female', 'male', 'male', 'male', 'male', 'ma...","['Lesbian', 'Pansexual', 'Homosexual', 'Hetero...","['Arab', 'White', 'Caucasian', 'South Asian', ...","['Poet', 'Game Developer', 'Research Scientist...","[""Master's"", 'Some College', ""Bachelor's"", 'Ph...","@FitnessJunkie07, I agree, but sometimes chang..."
2,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I completely agree that resp...","['Community-oriented', 'Community-oriented', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2]","['58', '58', '58', '58', '58', '58', '58', '58...","['female', 'female', 'female', 'female', 'fema...","['Heterosexual', 'Heterosexual', 'Heterosexual...","['Hispanic', 'Hispanic', 'Hispanic', 'Hispanic...","['Nonprofit Organizer', 'Nonprofit Organizer',...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","[(20.956, 32.0], (20.956, 32.0], (20.956, 32.0...","['female', 'male', 'male', 'male', 'male', 'ma...","['Lesbian', 'Pansexual', 'Homosexual', 'Hetero...","['Arab', 'White', 'Caucasian', 'South Asian', ...","['Poet', 'Game Developer', 'Research Scientist...","[""Master's"", 'Some College', ""Bachelor's"", 'Ph...","@FitnessJunkie07, I completely agree that resp..."
3,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I understand where you're co...","['Community-oriented', 'Community-oriented', '...","[1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2]","[2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2]","['58', '58', '58', '58', '58', '58', '58', '58...","['female', 'female', 'female', 'female', 'fema...","['Heterosexual', 'Heterosexual', 'Heterosexual...","['Hispanic', 'Hispanic', 'Hispanic', 'Hispanic...","['Nonprofit Organizer', 'Nonprofit Organizer',...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","[(20.956, 32.0], (20.956, 32.0], (20.956, 32.0...","['female', 'male', 'male', 'male', 'male', 'ma...","['Lesbian', 'Pansexual', 'Homosexual', 'Hetero...","['Arab', 'White', 'Caucasian', 'South Asian', ...","['Poet', 'Game Developer', 'Research Scientist...","[""Master's"", 'Some College', ""Bachelor's"", 'Ph...","@FitnessJunkie07, I understand where you're co..."
4,028f869d-cf12-444a-aefc-62f8b6294acd,@LoyalVolunteer54 @FitnessJunkie07 It's import...,"['Moderator', 'Mod

In [3]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="conv_id",
    sdb_column="age_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/140 [00:00<?, ?it/s]

,028f869d-cf12-444a-aefc-62f8b6294acd,0594f5fe-3bef-4642-8bb7-9ac356f44acb,05d53124-194a-4608-9614-c7075f71a827,07bd0dbc-5591-4b1a-ba75-fb7a84c28986,095f8e2c-4c6b-45d3-983b-c0a836cd1d4e,0d4caeec-0d56-4074-a2df-4bf5012d1c19,11d555f4-f5e4-4f64-a0cd-f89f2b29553d,1257a767-cba5-43c7-a532-deda418022e5,139dce53-526b-4950-83cc-1ce060739e3e,15f0ac31-1302-40eb-ba03-b486561cfd2b,...,ed709c3a-29f6-45f3-a63b-f1f4283d3d89,f4c92f48-1045-49db-9ad5-c5aee0f650cc,f66c6907-aa30-4dc3-8752-fa4b6d926e14,f7221fe4-3eac-42f2-937a-1dacbbe72395,f8085173-8cbf-4c15-a9e8-713c40539bdd,f83f95e7-1200-4dcd-a620-29694858190b,fac8059e-390f-421a-bbb8-a4b615da8d35,fce13b53-ae50-47b2-b70e-1de66f31a8bf,fdb670f7-dd7e-4222-8f5e-e3c6a8fccff4,fe858614-6571-43e7-badf-3cdcb38337b3
"(20.956, 32.0]",1.0,1.0,1.0,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.780101,1.0,1.000000,1.0000
"(32.0, 43.0]",0.0,1.0,1.0,0.005701,1.0,0.536127,1.0,1.0,0.574828,1.000000,...,0.014721,1.0,0.106251,1.000000,1.000000,1.000000,1.000000,1.0,0.559325,1.0000
"(43.0, 54.0]",0.0,1.0,1.0,1.000000,1.0,1.000000,1.0,1.0,1.000000,0.844085,...,0.948978,1.0,0.971899,1.000000,1.000000,0.130197,1.000000,1.0,0.075346,1.0000
"(54.0, 65.0]",1.0,1.0,1.0,0.967057,1.0,1.000000,1.0,1.0,1.000000,1.000000,...,1.000000,1.0,1.000000,0.093679,0.351339,0.355459,1.000000,1.0,1.000000,0.9735


In [6]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="conv_id",
    sdb_column="sexual_orientation_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/140 [00:00<?, ?it/s]

AssertionError: values [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] (length 11) 


In [5]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="conv_id",
    sdb_column="demographic_group_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/140 [00:00<?, ?it/s]

,028f869d-cf12-444a-aefc-62f8b6294acd,0594f5fe-3bef-4642-8bb7-9ac356f44acb,05d53124-194a-4608-9614-c7075f71a827,07bd0dbc-5591-4b1a-ba75-fb7a84c28986,095f8e2c-4c6b-45d3-983b-c0a836cd1d4e,0d4caeec-0d56-4074-a2df-4bf5012d1c19,11d555f4-f5e4-4f64-a0cd-f89f2b29553d,1257a767-cba5-43c7-a532-deda418022e5,139dce53-526b-4950-83cc-1ce060739e3e,15f0ac31-1302-40eb-ba03-b486561cfd2b,...,ed709c3a-29f6-45f3-a63b-f1f4283d3d89,f4c92f48-1045-49db-9ad5-c5aee0f650cc,f66c6907-aa30-4dc3-8752-fa4b6d926e14,f7221fe4-3eac-42f2-937a-1dacbbe72395,f8085173-8cbf-4c15-a9e8-713c40539bdd,f83f95e7-1200-4dcd-a620-29694858190b,fac8059e-390f-421a-bbb8-a4b615da8d35,fce13b53-ae50-47b2-b70e-1de66f31a8bf,fdb670f7-dd7e-4222-8f5e-e3c6a8fccff4,fe858614-6571-43e7-badf-3cdcb38337b3
'Arab',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Black',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Caucasian',1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.185978,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
'Indigenous',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Latino',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Middle Eastern',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Pacific Islander',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'South Asian',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'White',1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.974623,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
